In [1]:
import tkinter as tk
from tkinter import filedialog

# using datetime module
import datetime;

# ct stores current time
ct = datetime.datetime.now()
print(f"current time: {ct}")

# ts store timestamp of current time
ts = ct.timestamp()
print(f"timestamp: {ts}")

def show_save_window():
    save_win = tk.Toplevel()
    save_win.title("Messaggio")

    messaggio = tk.Label(save_win, text="Save current simulation on file 'sim_data.log'?")
    messaggio.pack(pady=10)

    def save_sim_data():
        print("Pippo")
        save_win.destroy()

    def skip():
        save_win.destroy()

    # Aggiungi i pulsanti
    save_button = tk.Button(save_win, text="Yes", command=stampa_pippo)
    save_button.pack(side=tk.LEFT, padx=10, pady=10)

    skip_button = tk.Button(save_win, text="No", command=skip)
    skip_button.pack(side=tk.RIGHT, padx=10, pady=10)

def upload_file():
    # Apri il file dialog per scegliere un file
    file_path = filedialog.askopenfilename(title="Seleziona un file")
    if file_path:
        try:
            # Leggi il file e stampalo riga per riga
            with open(file_path, 'r') as file:
                for line in file:
                    print(line.strip())
        except Exception as e:
            print(f"Errore durante la lettura del file: {e}")

# Crea la finestra principale
#root = tk.Tk()
#root.title("Esempio di Tkinter")

# Pulsante per mostrare il messaggio
#bottone_mostra = tk.Button(root, text="Mostra messaggio", command=mostra_messaggio)
#bottone_mostra.pack(pady=20)

# Pulsante per caricare un file
#bottone_upload = tk.Button(root, text="Upload File", command=upload_file)
#bottone_upload.pack(pady=20)

# Avvia il loop principale dell'interfaccia grafica
#root.mainloop()


scenarios = scen.generate_scenarios_from_file("sim_data_all.log")




current time: 2024-12-16 19:20:24.163863
timestamp: 1734373224.163863
